In [1]:
import pint
ureg = pint.UnitRegistry()
# ureg.define('dollar = dimensionless')
# dollar = ureg.Quantity(1, 'dollar')
dollar = 1

In [2]:
# Status Quo (Diesel)
# source: https://www.thetruckersreport.com/infographics/cost-of-trucking/
# alternate source: https://www.freightwaves.com/news/understanding-total-operating-cost-per-mile#:~:text=Typically%2C%20gross%20fuel%20expense%20averages%20between%20%240.40%2D%240.55%20per%20mile.
per_mile_us_shipping_cost = 1.38 * dollar / (ureg.mile)
per_mile_fuel_cost = 0.54 * dollar / (ureg.mile)
per_mile_equipment_cost = 0.24 * dollar / (ureg.mile)
per_mile_driver_cost = 0.36 * dollar / (ureg.mile)
per_mile_maintenance_cost = 0.12 * dollar / (ureg.mile)
per_mile_insurance_cost = 0.05 * dollar / (ureg.mile)


# Status Quo (Electric)

# source: https://apps.dana.com/commercial-vehicles/tco/
# alt source: https://avt.inl.gov/sites/default/files/pdf/fsev/costs.pdf
per_mile_electricity_cost = 0.45 * per_mile_fuel_cost

# source: https://apps.dana.com/commercial-vehicles/tco/
per_mile_electric_maintenance_cost = 0.3 * per_mile_maintenance_cost

# based on Tesla semi
# source: https://electrek.co/2018/05/02/tesla-semi-production-version-range-increase-elon-musk/
# alt source: https://lynceans.org/wp-content/uploads/2020/04/Tesla-Semi-converted.pdf
# Important note - this says a Tesla semi will have ~20% lower operating costs, so a partial
# electric conversion will be 
per_mile_electric_energy_use = 900 * ureg.kWh / (600 * ureg.mile) 

In [3]:
# Proposal for first route, since LCVs are legal on this route and it has high volume
# source: https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/nhsavglhft2015.htm
pgh_to_chi = 431 * ureg.miles

In [4]:
# Best battery for application: Lithium Iron Phosphate
# source: https://batteryuniversity.com/learn/article/types_of_lithium_ion
# High cycles
# Cheap
# Good low temp performance
# Safe (not nearly as suscpetible to thermal runaway as compared to Cobalt-based cathodes in Tesla Semi)

# source: https://www.alibaba.com/product-detail/Graphene-lithium-iron-phosphate-battery-3_1600131559793.html
battery_price_per_kwh = (63 * dollar / (272 * ureg.amp * ureg.hour * 3.2 * ureg.volt)).to('1/kWh')
battery_lifespan = 10000 * pgh_to_chi
depletion_factor = 0.8 # how much capacity it has after full cycles
lfp_density = (272 * ureg.amp * ureg.hour * 3.2 * ureg.volt) / (5.4 * ureg.kg)

In [5]:
# Parameters that can vary. Fiddle with these to test different operating conditions\
# If you want to test different configurations, these variables can be changed

# How much work (proportionally) the electric motors will do (1 means 0% diesel, 0 means 100% diesel)
proportion_to_electric = 0.7

# How much of the cost savings we claim as revenue, the rest passed on to the rest of the market
# (carriers, owner/operators, etc)
prop_take = 0.4

# How much more the tolls will cost by adding a 1-axle vehicle to the train
# source: https://www.ohioturnpike.org/docs/default-source/annual-report-files/cafr-2019-final.pdf?sfvrsn=d361ebc4_2
# when >3 trailers, ie classified as a road train (A type, ie fifth wheel to fifth wheel, 53' trailers)
# per_mile_toll_increase = 0.07 * dollar / ureg.mile
# when no increase in trailers, ie classified as a road train (B type, ie tow hitch to fifth wheel, twin 28' trailer LTL market)
per_mile_toll_increase = 0.0 * dollar / ureg.mile

 # proportion of day on road
sixth_wheel_utilization = 0.6

# avg speed when on road
sixth_wheel_speed = 45 * ureg.mph

# proportional cost of components (as a proportion of battery cost), roughly based on Tesla
additional_component_prop_cost = 0.1
# proportional weight of components (as a proportion of battery cost), roughly based on Tesla
additional_component_prop_weight = 0.25

# margins of sixth wheel operation (not including driving operating costs, mainly yard things like recharding)
sixth_wheel_margins = 0.6

In [6]:
# iteratively get capacity, since more battery weight means need for more capacity
added_weight_factor = 1.18
resultant_weight_factor = 1

# https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/docs/10factsfigures/table3_3.htm
# https://www.freightpros.com/blog/less-than-truckload/
# https://www.terrybryant.com/how-much-does-semi-truck-weigh#:~:text=Semi%2Dtractors%20weigh%20up%20to,haul%20up%20to%2034%2C000%20pounds.
base_truck_weight = 68000 * ureg.pound
percent_under_base_weight = 0.7

while abs(added_weight_factor - resultant_weight_factor) > 0.00001:
    added_weight_factor = resultant_weight_factor
    battery_capacity = proportion_to_electric * per_mile_electric_energy_use * pgh_to_chi * added_weight_factor / depletion_factor
    battery_price = battery_price_per_kwh * battery_capacity

    lfp_battery_weight = battery_capacity / lfp_density

    resultant_weight_factor = (base_truck_weight + lfp_battery_weight*(1+additional_component_prop_weight)) / base_truck_weight
    
added_weight_factor = resultant_weight_factor
    
print("total truck weight (must be <80k lbs)", (base_truck_weight*added_weight_factor).to('lb'))

total truck weight (must be <80k lbs) 79275.10399105446 pound


In [7]:
# operating costs, averaged as a funtion of the proportion converted to electricity
sixth_wheel_per_mile_fuel_cost = (per_mile_fuel_cost * (1-proportion_to_electric) + \
                                    per_mile_electricity_cost * (proportion_to_electric)) * added_weight_factor
sixth_wheel_per_mile_maintenance_cost = (per_mile_maintenance_cost * (1-proportion_to_electric) + \
                                    per_mile_electric_maintenance_cost * (proportion_to_electric)) * added_weight_factor
# Assume this is slightly lower because of ADAS features like ABS, ACC, anti-skid
sixth_wheel_per_mile_insurance_cost = 0.8 * per_mile_insurance_cost

# PRICING
delta = -per_mile_toll_increase + (per_mile_fuel_cost + per_mile_maintenance_cost + per_mile_insurance_cost) - \
        (sixth_wheel_per_mile_fuel_cost + sixth_wheel_per_mile_maintenance_cost + sixth_wheel_per_mile_insurance_cost)
sixth_wheel_per_mile_rental_cost = prop_take * delta

In [8]:

 # proportion of day on road# Total cost savings

sixth_wheel_per_mile_cost_no_cut = per_mile_us_shipping_cost - delta

overall_savings_no_cut = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost_no_cut) / per_mile_us_shipping_cost * 100
driver_savings_no_cut = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost_no_cut) / per_mile_driver_cost * 100
print("Overall percentage savings (WITHOUT OUR CUT) %", overall_savings_no_cut)
print("Potential driver wage increase (if given only to driver, WITHOUT OUR CUT) %", driver_savings_no_cut)

sixth_wheel_per_mile_cost = per_mile_us_shipping_cost - delta + \
                            + sixth_wheel_per_mile_rental_cost

overall_savings = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost) / per_mile_us_shipping_cost * 100
driver_savings = (per_mile_us_shipping_cost - sixth_wheel_per_mile_cost) / per_mile_driver_cost * 100

print("Overall percentage savings (WITH OUR CUT) %", overall_savings)
print("Potential driver wage increase (if given only to driver, WITH OUR CUT) %", driver_savings)

Overall percentage savings (WITHOUT OUR CUT) % 15.325129582606865 dimensionless
Potential driver wage increase (if given only to driver, WITHOUT OUR CUT) % 58.74633006665964 dimensionless
Overall percentage savings (WITH OUR CUT) % 9.19507774956412 dimensionless
Potential driver wage increase (if given only to driver, WITH OUR CUT) % 35.24779803999579 dimensionless


In [13]:
#  Capital analysis

sixth_wheel_capital_expense = battery_price * (1+additional_component_prop_cost)
sixth_wheel_miles_per_year = (sixth_wheel_utilization * sixth_wheel_speed * ureg.year).to('mile')
sixth_wheel_annual_revenue = (sixth_wheel_miles_per_year * sixth_wheel_per_mile_rental_cost).to('dimensionless')

print("Cost of each sixth wheel", sixth_wheel_capital_expense)
print("lifetime value", sixth_wheel_per_mile_rental_cost * sixth_wheel_miles_per_year * (battery_lifespan / sixth_wheel_miles_per_year) * sixth_wheel_margins)
print("sixth wheel payback period", ureg.year * sixth_wheel_capital_expense / (sixth_wheel_annual_revenue * sixth_wheel_margins))
print("sixth wheel (rough) capital efficiency (%)", sixth_wheel_annual_revenue * sixth_wheel_margins/ (sixth_wheel_capital_expense ) * 100)

# source: https://ops.fhwa.dot.gov/freight/freight_analysis/nat_freight_stats/nhsavglhft2015.htm
pgh_to_chi_daily_trucks = 25000 
pgh_to_chi_tam_no_yard = 365 * pgh_to_chi_daily_trucks * pgh_to_chi * sixth_wheel_per_mile_rental_cost
print("PGH to CHI no yard TAM (not including split between our take and that given to market) ($)", percent_under_base_weight * pgh_to_chi_tam_no_yard)

# source: https://www.commerceexpressinc.com/2020/01/09/5-quick-facts-about-semi-trucks/#:~:text=Engines%20can't%20last%20forever,%2Dtruck%2015%2D16%20years.
# source: https://hdstruckdrivinginstitute.com/semi-trucks-numbers/#:~:text=There%20are%20nearly%202%20million,tractors%20and%20tractor%20trailers%2C%20respectively.
annual_semi_miles = 140e9*ureg.miles
print("Total US trucking no yard TAM (not including split between our take and that given to market) ($)", percent_under_base_weight * annual_semi_miles * sixth_wheel_per_mile_rental_cost)

Cost of each sixth wheel 52506.82505066826 dimensionless
lifetime value 218761.93375542984 dimensionless
sixth wheel payback period 4.370750786972369 year
sixth wheel (rough) capital efficiency (%) 22.87936440990045 dimensionless
PGH to CHI no yard TAM (not including split between our take and that given to market) ($) 232890308.64380136 dimensionless
Total US trucking no yard TAM (not including split between our take and that given to market) ($) 8290282099.007009 dimensionless


In [14]:
#  Capital analysis, other revenue streams
#  Yard operations (using sixth wheel as a yard jockey)
#  source: baton.io
detention_time_per_trip = 4 * ureg.hours
percent_time_improvement = 0.7
trips_per_year = pgh_to_chi_daily_trucks * 365 * percent_time_improvement
hourly_wages = 24.48 * dollar / ureg.hour # source: https://www.bls.gov/ooh/transportation-and-material-moving/material-moving-machine-operators.htm
pgh_to_chi_tam_yard = detention_time_per_trip * hourly_wages * trips_per_year
# NOTE: making the sixth wheels autonomous yard jockeys will also improve margins on this market
print("PGH to CHI yard TAM", percent_under_base_weight * pgh_to_chi_tam_yard)

average_trip_length = 500 * ureg.miles
print("Total US trucking yard TAM", percent_under_base_weight * detention_time_per_trip * hourly_wages * (annual_semi_miles / average_trip_length))

sixth_wheel_per_mile_rental_cost_with_yard = sixth_wheel_per_mile_rental_cost + detention_time_per_trip * hourly_wages / average_trip_length
sixth_wheel_annual_revenue_with_yard = (sixth_wheel_miles_per_year * sixth_wheel_per_mile_rental_cost_with_yard).to('dimensionless')

print("lifetime value w/ yard", sixth_wheel_per_mile_rental_cost_with_yard * sixth_wheel_miles_per_year * (battery_lifespan / sixth_wheel_miles_per_year) * sixth_wheel_margins)
print("sixth wheel payback period w/ yard", ureg.year * sixth_wheel_capital_expense / (sixth_wheel_annual_revenue_with_yard * sixth_wheel_margins))
print("sixth wheel (rough) capital efficiency  w/ yard (%)", sixth_wheel_annual_revenue_with_yard * sixth_wheel_margins/ (sixth_wheel_capital_expense ) * 100)


PGH to CHI yard TAM 437824800.0 dimensionless
Total US trucking yard TAM 19192320000.0 dimensionless
lifetime value w/ yard 725204.1737554298 dimensionless
sixth wheel payback period w/ yard 1.3184616535916387 year
sixth wheel (rough) capital efficiency  w/ yard (%) 75.84596770606767 dimensionless


In [12]:
#  Capital analysis, other revenue streams
#  Autonomy (when full autonomy is achieved)
sixth_wheel_per_mile_driver_cost_autonomy = 0.15 * dollar / (ureg.mile) # software license fee to AV company

delta_autonomy = -per_mile_toll_increase + (per_mile_fuel_cost + per_mile_maintenance_cost + per_mile_insurance_cost + per_mile_driver_cost) - \
        (sixth_wheel_per_mile_fuel_cost + sixth_wheel_per_mile_maintenance_cost + sixth_wheel_per_mile_insurance_cost + sixth_wheel_per_mile_driver_cost_autonomy)
sixth_wheel_per_mile_rental_cost_autonomy = prop_take * delta_autonomy

sixth_wheel_annual_revenue_autonomy = (365 * pgh_to_chi_daily_trucks * pgh_to_chi * sixth_wheel_per_mile_rental_cost_autonomy).to('dimensionless')
print("PGH to CHI annual revenue, autonomy", sixth_wheel_annual_revenue_autonomy)

print("Total US trucking annual revenue, autonomy", annual_semi_miles * sixth_wheel_per_mile_rental_cost_autonomy)

sixth_wheel_per_mile_rental_cost_with_yard_autonomy = sixth_wheel_per_mile_rental_cost_autonomy + detention_time_per_trip * hourly_wages / average_trip_length
sixth_wheel_annual_revenue_with_yard_autonomy = (sixth_wheel_miles_per_year * sixth_wheel_per_mile_rental_cost_with_yard_autonomy).to('dimensionless')

print("lifetime value  w/ yard and AV", sixth_wheel_per_mile_rental_cost_with_yard_autonomy * sixth_wheel_miles_per_year * (battery_lifespan / sixth_wheel_miles_per_year) * sixth_wheel_margins)
print("sixth wheel payback period  w/ yard and AV", ureg.year * sixth_wheel_capital_expense / (sixth_wheel_annual_revenue_with_yard_autonomy * sixth_wheel_margins))
print("sixth wheel (rough) capital efficiency  w/ yard and AV (%)", sixth_wheel_annual_revenue_with_yard_autonomy * sixth_wheel_margins/ (sixth_wheel_capital_expense ) * 100)


PGH to CHI annual revenue, autonomy 663061940.9197162 dimensionless
Total US trucking annual revenue, autonomy 23603260141.438583 dimensionless
lifetime value  w/ yard and AV 942428.1737554298 dimensionless
sixth wheel payback period  w/ yard and AV 1.014564208443618 year
sixth wheel (rough) capital efficiency  w/ yard and AV (%) 98.56448627672762 dimensionless
